In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [2]:
def normalize_data(data):
    mean = torch.mean(data, dim=0, keepdim=True)
    std = torch.std(data, dim=0, keepdim=True)
    std = torch.clamp(std, min=1e-8)
    return (data - mean) / std

In [3]:
def cal_sample_weight(labels, num_class, use_sample_weight=True):
    if not use_sample_weight:
        return np.ones(len(labels)) / len(labels)
    count = np.zeros(num_class)
    for i in range(num_class):
        count[i] = np.sum(labels==i)
    sample_weight = np.zeros(labels.shape)
    for i in range(num_class):
        sample_weight[np.where(labels==i)[0]] = count[i]/np.sum(count)
    
    return sample_weight

In [4]:
def one_hot_tensor(y, num_dim):
    y_onehot = torch.zeros(y.shape[0], num_dim)
    y_onehot.scatter_(1, y.view(-1,1), 1)
    
    return y_onehot

In [5]:
def to_sparse(x):
    x_typename = torch.typename(x).split('.')[-1]
    sparse_tensortype = getattr(torch.sparse, x_typename)
    indices = torch.nonzero(x)
    if len(indices.shape) == 0:  # if all elements are zeros
        return sparse_tensortype(*x.shape)
    indices = indices.t()
    values = x[tuple(indices[i] for i in range(indices.shape[0]))]
    return sparse_tensortype(indices, values, x.size())

In [6]:
def rbf_kernel(x1, x2=None, gamma=None):
    """
    Radial Basis Function (Gaussian) Kernel as similarity.
    """
    print("From rbf_kernel")
    x2 = x1 if x2 is None else x2
    diff = x1.unsqueeze(1) - x2.unsqueeze(0)  # [N, M, D]
    dist_sq = torch.sum(diff ** 2, dim=2)     # [N, M]
    if gamma is None:
        gamma = 1.0 / x1.shape[1]             # Default: 1 / num_features
    sim = torch.exp(-gamma * dist_sq)
    return sim

In [7]:
def adaptive_rbf_distance_torch(x1, x2=None):
    print("From adaptive_rbf_distance_torch")
    x2 = x1 if x2 is None else x2
    # Compute median pairwise distance
    x1_norm = (x1 ** 2).sum(dim=1).view(-1, 1)
    x2_norm = x1_norm if x2 is x1 else (x2 ** 2).sum(dim=1).view(1, -1)
    dist_squared = x1_norm + x2_norm - 2.0 * torch.mm(x1, x2.T)
    median_dist = torch.median(dist_squared[dist_squared > 0])
    gamma = 1.0 / (2.0 * median_dist)  # Adaptive gamma
    sim = torch.exp(-gamma * dist_squared)
    print("Sim: ",sim)
    return 1 - sim

In [8]:
def cosine_distance_torch(x1, x2=None, eps=1e-8):
    x2 = x1 if x2 is None else x2
    w1 = x1.norm(p=2, dim=1, keepdim=True)
    w2 = w1 if x2 is x1 else x2.norm(p=2, dim=1, keepdim=True)
    res = 1 - torch.mm(x1, x2.t()) / (w1 * w2.t()).clamp(min=eps)
    return res

In [10]:
def rbf_distance_torch(x1, x2=None, gamma=None):
    """
    Computes RBF-based pairwise distances between samples.
    The smaller the Euclidean distance, the higher the similarity (max=1).
    """
    print("From rbf_distance_torch")
    x2 = x1 if x2 is None else x2
    x1_norm = (x1 ** 2).sum(dim=1).view(-1, 1)
    x2_norm = x1_norm if x2 is x1 else (x2 ** 2).sum(dim=1).view(1, -1)
    
    dist_squared = x1_norm + x2_norm - 2.0 * torch.mm(x1, x2.T)
    dist_squared = torch.clamp(dist_squared, min=0.0)  # numerical safety
    
    if gamma is None:
        median_dist = torch.median(dist_squared[dist_squared > 0])
        gamma = 1.0 / (2.0 * median_dist)

    sim = torch.exp(-gamma * dist_squared)  # RBF similarity
    res = 1 - sim  # Return distance (small similarity -> larger distance)
    return res

In [11]:
def hybrid_similarity_torch(x1, x2=None, alpha=0.5, gamma=0.01):
    print("Hybrid")
    x2 = x1 if x2 is None else x2
    
    # Calculate cosine similarity
    cosine_sim = cosine_distance_torch(x1, x2)
    
    # Normalize cosine similarity to be in range [0, 1]
    cosine_sim = torch.clamp(1 - cosine_sim, 0, 1)  # Cosine similarity is between 0 and 1
    print("COS: ")
    print(cosine_sim[:5, :5])
    # Calculate RBF similarity
    rbf_sim = rbf_distance_torch(x1, x2, gamma=gamma)
    
    # Normalize RBF similarity to be in range [0, 1] (if not already)
    rbf_sim = torch.clamp(rbf_sim, 0, 1)
    print("RBF: ")
    print(rbf_sim[:5, :5])
    alpha = cosine_sim.var() / (cosine_sim.var() + rbf_sim.var() + 1e-8)
    print("Alpha: ", alpha)
    # Combine similarities
    hybrid_sim = alpha * cosine_sim + (1 - alpha) * rbf_sim
    
    # Ensure the hybrid similarity is within the range [0, 1]
    hybrid_sim = torch.clamp(hybrid_sim, 0, 1)
    hybrid_sim = F.normalize(hybrid_sim, p=1, dim=1)

    print("\nHybrid Similarity Matrix Shape:", hybrid_sim.shape)
    print("Hybrid Similarity Values (first 5x5):")
    print(hybrid_sim[:5, :5])
    
    return hybrid_sim

In [12]:
# def cal_adj_mat_parameter(edge_per_node, data, metric="rbf"):
# def cal_adj_mat_parameter(edge_per_node, data, metric="cosine"):
# def cal_adj_mat_parameter(edge_per_node, data, metric="adaptive_rbf"):
def cal_adj_mat_parameter(edge_per_node, data, metric="hybrid"):
    assert metric in ["rbf", "cosine", "adaptive_rbf", "hybrid"], "Only rbf, cosine, adaptive_rbf, and hybrid supported"

    if metric == "rbf":
        dist = rbf_distance_torch(data, data, gamma=0.01)
    elif metric == "cosine":
        dist = cosine_distance_torch(data, data)
    elif metric == "adaptive_rbf":
        dist = adaptive_rbf_distance_torch(data, data)
    elif metric == "hybrid":
        dist = hybrid_similarity_torch(data, data, alpha=0.5, gamma=0.01)

    parameter = torch.sort(dist.reshape(-1,)).values[edge_per_node*data.shape[0]]
    return np.isscalar(parameter.data.cpu().numpy())
    #return parameter.item()  # Return the scalar value

In [13]:
def graph_from_dist_tensor(dist, parameter, self_dist=True):
    if self_dist:
        assert dist.shape[0]==dist.shape[1], "Input is not pairwise dist matrix"
    g = (dist <= parameter).float()
    if self_dist:
        diag_idx = np.diag_indices(g.shape[0])
        g[diag_idx[0], diag_idx[1]] = 0
    return g

In [14]:
def graph_from_topk_tensor(similarity, k):
    topk_values, topk_indices = torch.topk(similarity, k=k, dim=-1)
    mask = torch.zeros_like(similarity)
    mask.scatter_(1, topk_indices, 1)
    return mask

In [15]:
# def gen_adj_mat_tensor(data, parameter, metric="rbf"):
# def gen_adj_mat_tensor(data, parameter, metric="cosine"):
# def gen_adj_mat_tensor(data, parameter, metric="adaptive_rbf"):
def gen_adj_mat_tensor(data, parameter, metric="hybrid"):
    assert metric in ["rbf", "cosine", "adaptive_rbf", "hybrid"], "Only rbf, cosine, adaptive_rbf, and hybrid supported"

    # Normalize data before computing similarity
    data = normalize_data(data)

    if metric == "rbf":
        dist = rbf_distance_torch(data, data, gamma=0.01)
    elif metric == "cosine":
        dist = cosine_distance_torch(data, data)
    elif metric == "adaptive_rbf":
        dist = adaptive_rbf_distance_torch(data, data)
    elif metric == "hybrid":
        dist = hybrid_similarity_torch(data, data, alpha=0.5, gamma=0.01)

    g = graph_from_dist_tensor(dist, parameter, self_dist=True)
    if metric in ["rbf", "cosine", "adaptive_rbf", "hybrid"]:
        adj = 1-dist
    else:
        adj = 1-dist
    adj = adj*g 
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda()
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    
    # Convert to dense tensor before returning
    adj = adj.to_dense()
    
    return adj

In [16]:
# def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="rbf"):
# def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="cosine"):
# def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="adaptive_rbf"):
def gen_test_adj_mat_tensor(data, trte_idx, parameter, metric="hybrid"):
    assert metric in ["rbf", "cosine", "adaptive_rbf", "hybrid"], "Only rbf, cosine, adaptive_rbf, and hybrid supported"

    # Normalize data before computing similarity
    data = normalize_data(data)

    adj = torch.zeros((data.shape[0], data.shape[0]))
    if cuda:
        adj = adj.cuda()
    num_tr = len(trte_idx["tr"])
    
    if metric == "rbf":
        dist_tr2te = rbf_distance_torch(data[trte_idx["tr"]], data[trte_idx["te"]], gamma=0.01)
    elif metric == "cosine":
        dist_tr2te = cosine_distance_torch(data[trte_idx["tr"]], data[trte_idx["te"]])
    elif metric == "adaptive_rbf":
        dist_tr2te = adaptive_rbf_distance_torch(data[trte_idx["tr"]], data[trte_idx["te"]])
    elif metric == "hybrid":
        dist_tr2te = hybrid_similarity_torch(data[trte_idx["tr"]], data[trte_idx["te"]], alpha=0.5)

    g_tr2te = graph_from_dist_tensor(dist_tr2te, parameter, self_dist=False)
    adj[:num_tr,num_tr:] = 1-dist_tr2te
    adj[:num_tr,num_tr:] = adj[:num_tr,num_tr:]*g_tr2te
    
    if metric == "rbf":
        dist_te2tr = rbf_distance_torch(data[trte_idx["te"]], data[trte_idx["tr"]], gamma=0.01)
    elif metric == "cosine":
        dist_te2tr = cosine_distance_torch(data[trte_idx["te"]], data[trte_idx["tr"]])
    elif metric == "adaptive_rbf":
        dist_te2tr = adaptive_rbf_distance_torch(data[trte_idx["te"]], data[trte_idx["tr"]])
    elif metric == "hybrid":
        dist_te2tr = hybrid_similarity_torch(data[trte_idx["te"]], data[trte_idx["tr"]], alpha=0.5)

    g_te2tr = graph_from_dist_tensor(dist_te2tr, parameter, self_dist=False)
    adj[num_tr:,:num_tr] = 1-dist_te2tr
    adj[num_tr:,:num_tr] = adj[num_tr:,:num_tr]*g_te2tr
    
    adj_T = adj.transpose(0,1)
    I = torch.eye(adj.shape[0])
    if cuda:
        I = I.cuda()
    adj = adj + adj_T*(adj_T > adj).float() - adj*(adj_T > adj).float()
    adj = F.normalize(adj + I, p=1)
    
    # Convert to dense tensor before returning
    adj = adj.to_dense()
    
    return adj

In [17]:
def save_model_dict(folder, model_dict):
    if not os.path.exists(folder):
        os.makedirs(folder)
    for module in model_dict:
        torch.save(model_dict[module].state_dict(), os.path.join(folder, module+".pth"))

In [18]:
def load_model_dict(folder, model_dict):
    for module in model_dict:
        if os.path.exists(os.path.join(folder, module+".pth")):
#            print("Module {:} loaded!".format(module))
            model_dict[module].load_state_dict(torch.load(os.path.join(folder, module+".pth"), map_location="cuda:{:}".format(torch.cuda.current_device())))
        else:
            print("WARNING: Module {:} from model_dict is not loaded!".format(module))
        if cuda:
            model_dict[module].cuda()    
    return model_dict

In [19]:
def xavier_init(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
           m.bias.data.fill_(0.0)

In [20]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        nn.init.xavier_normal_(self.weight.data)
        if self.bias is not None:
            self.bias.data.fill_(0.0)
    
    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.sparse.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

In [21]:
class GCN_E(nn.Module):
    def __init__(self, in_dim, hgcn_dim, dropout):
        super().__init__()
        self.gc1 = GraphConvolution(in_dim, hgcn_dim[0])
        self.gc2 = GraphConvolution(hgcn_dim[0], hgcn_dim[1])
        self.gc3 = GraphConvolution(hgcn_dim[1], hgcn_dim[2])
        self.dropout = dropout

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.leaky_relu(x, 0.25)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = F.leaky_relu(x, 0.25)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc3(x, adj)
        x = F.leaky_relu(x, 0.25)
        
        return x

In [22]:
class Classifier_1(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.clf = nn.Sequential(nn.Linear(in_dim, out_dim))
        self.clf.apply(xavier_init)

    def forward(self, x):
        x = self.clf(x)
        return x

In [23]:
class VCDN(nn.Module):
    def __init__(self, num_view, num_cls, hvcdn_dim):
        super().__init__()
        self.num_cls = num_cls
        self.model = nn.Sequential(
            nn.Linear(pow(num_cls, num_view), hvcdn_dim),
            nn.LeakyReLU(0.25),
            nn.Linear(hvcdn_dim, num_cls)
        )
        self.model.apply(xavier_init)
        
    def forward(self, in_list):
        num_view = len(in_list)
        for i in range(num_view):
            in_list[i] = torch.sigmoid(in_list[i])
        x = torch.reshape(torch.matmul(in_list[0].unsqueeze(-1), in_list[1].unsqueeze(1)),(-1,pow(self.num_cls,2),1))
        for i in range(2,num_view):
            x = torch.reshape(torch.matmul(x, in_list[i].unsqueeze(1)),(-1,pow(self.num_cls,i+1),1))
        vcdn_feat = torch.reshape(x, (-1,pow(self.num_cls,num_view)))
        output = self.model(vcdn_feat)

        return output

In [24]:
def init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hc, gcn_dopout=0.5):
    model_dict = {}
    for i in range(num_view):
        model_dict["E{:}".format(i+1)] = GCN_E(dim_list[i], dim_he_list, gcn_dopout)
        model_dict["C{:}".format(i+1)] = Classifier_1(dim_he_list[-1], num_class)
    if num_view >= 2:
        model_dict["C"] = VCDN(num_view, num_class, dim_hc)
    return model_dict


In [25]:
def init_optim(num_view, model_dict, lr_e=1e-4, lr_c=1e-4):
    optim_dict = {}
    for i in range(num_view):
        optim_dict["C{:}".format(i+1)] = torch.optim.Adam(
                list(model_dict["E{:}".format(i+1)].parameters())+list(model_dict["C{:}".format(i+1)].parameters()), 
                lr=lr_e)
    if num_view >= 2:
        optim_dict["C"] = torch.optim.Adam(model_dict["C"].parameters(), lr=lr_c)
    return optim_dict

In [26]:
def prepare_trte_data(data_folder, view_list):
    num_view = len(view_list)
    labels_tr = np.loadtxt(os.path.join(data_folder, "labels_tr.csv"), delimiter=',')
    labels_te = np.loadtxt(os.path.join(data_folder, "labels_te.csv"), delimiter=',')
    labels_tr = labels_tr.astype(int)
    labels_te = labels_te.astype(int)
    data_tr_list = []
    data_te_list = []
    for i in view_list:
        data_tr_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_tr.csv"), delimiter=','))
        data_te_list.append(np.loadtxt(os.path.join(data_folder, str(i)+"_te.csv"), delimiter=','))
    num_tr = data_tr_list[0].shape[0]
    num_te = data_te_list[0].shape[0]
    data_mat_list = []
    for i in range(num_view):
        data_mat_list.append(np.concatenate((data_tr_list[i], data_te_list[i]), axis=0))
    data_tensor_list = []
    for i in range(len(data_mat_list)):
        data_tensor_list.append(torch.FloatTensor(data_mat_list[i]))
        if cuda:
            data_tensor_list[i] = data_tensor_list[i].cuda()
    idx_dict = {}
    idx_dict["tr"] = list(range(num_tr))
    idx_dict["te"] = list(range(num_tr, (num_tr+num_te)))
    data_train_list = []
    data_all_list = []
    for i in range(len(data_tensor_list)):
        data_train_list.append(data_tensor_list[i][idx_dict["tr"]].clone())
        data_all_list.append(torch.cat((data_tensor_list[i][idx_dict["tr"]].clone(),
                                       data_tensor_list[i][idx_dict["te"]].clone()),0))
    labels = np.concatenate((labels_tr, labels_te))
    
    return data_train_list, data_all_list, idx_dict, labels

In [27]:
def gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter):
    #adj_metric = "cosine" # cosine distance
    #adj_metric = "rbf"
    #adj_metric = "adaptive_rbf"
    adj_metric = "hybrid"
    adj_train_list = []
    adj_test_list = []
    
    # Create directory for adjacency matrices if it doesn't exist
    adj_dir = "adjacency_matrices"
    if not os.path.exists(adj_dir):
        os.makedirs(adj_dir)
    
    for i in range(len(data_tr_list)):
        adj_parameter_adaptive = cal_adj_mat_parameter(adj_parameter, data_tr_list[i], adj_metric)
        adj_train = gen_adj_mat_tensor(data_tr_list[i], adj_parameter_adaptive, adj_metric)
        adj_test = gen_test_adj_mat_tensor(data_trte_list[i], trte_idx, adj_parameter_adaptive, adj_metric)
        
        # Convert to numpy arrays
        adj_train_np = adj_train.cpu().numpy()
        adj_test_np = adj_test.cpu().numpy()
        
        # Save to CSV files
        np.savetxt(os.path.join(adj_dir, f"view_{i+1}_train_adj.csv"), adj_train_np, delimiter=',')
        np.savetxt(os.path.join(adj_dir, f"view_{i+1}_test_adj.csv"), adj_test_np, delimiter=',')
        
        adj_train_list.append(adj_train)
        adj_test_list.append(adj_test)
    
    print(f"\nAdjacency matrices saved to {adj_dir} directory:")
    for i in range(len(data_tr_list)):
        print(f"View {i+1}:")
        print(f"- Training adjacency matrix: view_{i+1}_train_adj.csv")
        print(f"- Test adjacency matrix: view_{i+1}_test_adj.csv")
    
    return adj_train_list, adj_test_list

In [28]:
def train_epoch(data_list, adj_list, label, one_hot_label, sample_weight, model_dict, optim_dict, train_VCDN=True):
    loss_dict = {}
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    for m in model_dict:
        model_dict[m].train()    
    num_view = len(data_list)
    for i in range(num_view):
        optim_dict["C{:}".format(i+1)].zero_grad()
        ci_loss = 0
        ci = model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i]))
        ci_loss = torch.mean(torch.mul(criterion(ci, label),sample_weight))
        ci_loss.backward()
        optim_dict["C{:}".format(i+1)].step()
        loss_dict["C{:}".format(i+1)] = ci_loss.detach().cpu().numpy().item()
    if train_VCDN and num_view >= 2:
        optim_dict["C"].zero_grad()
        c_loss = 0
        ci_list = []
        for i in range(num_view):
            ci_list.append(model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i])))
        c = model_dict["C"](ci_list)    
        c_loss = torch.mean(torch.mul(criterion(c, label),sample_weight))
        c_loss.backward()
        optim_dict["C"].step()
        loss_dict["C"] = c_loss.detach().cpu().numpy().item()
    
    return loss_dict

In [29]:
def test_epoch(data_list, adj_list, te_idx, model_dict):
    for m in model_dict:
        model_dict[m].eval()
    num_view = len(data_list)
    ci_list = []
    # Get predictions for each view
    for i in range(num_view):
        ci = model_dict["C{:}".format(i+1)](model_dict["E{:}".format(i+1)](data_list[i],adj_list[i]))
        ci_list.append(ci)
        # Print individual view predictions
        view_prob = F.softmax(ci[te_idx,:], dim=1).data.cpu().numpy()
        # print(f"\nView {i+1} predictions:")
        # print("Predicted classes:", view_prob.argmax(1))
        # print("Prediction probabilities:", view_prob)
    
    # Get combined prediction
    if num_view >= 2:
        c = model_dict["C"](ci_list)    
    else:
        c = ci_list[0]
    c = c[te_idx,:]
    prob = F.softmax(c, dim=1).data.cpu().numpy()
    
    return prob

In [30]:
def train_test(data_folder, view_list, num_class,
               lr_e_pretrain, lr_e, lr_c, 
               num_epoch_pretrain, num_epoch):
    test_inverval = 20
    num_view = len(view_list)
    dim_hvcdn = pow(num_class,num_view)
    if data_folder == 'ROSMAP':
        adj_parameter = 2
        dim_he_list = [200,200,100]
    if data_folder == 'BRCA':
        adj_parameter = 10
        dim_he_list = [400,400,200]
    data_tr_list, data_trte_list, trte_idx, labels_trte = prepare_trte_data(data_folder, view_list)
    labels_tr_tensor = torch.LongTensor(labels_trte[trte_idx["tr"]])
    onehot_labels_tr_tensor = one_hot_tensor(labels_tr_tensor, num_class)
    sample_weight_tr = cal_sample_weight(labels_trte[trte_idx["tr"]], num_class)
    sample_weight_tr = torch.FloatTensor(sample_weight_tr)
    if cuda:
        labels_tr_tensor = labels_tr_tensor.cuda()
        onehot_labels_tr_tensor = onehot_labels_tr_tensor.cuda()
        sample_weight_tr = sample_weight_tr.cuda()
    adj_tr_list, adj_te_list = gen_trte_adj_mat(data_tr_list, data_trte_list, trte_idx, adj_parameter)
    dim_list = [x.shape[1] for x in data_tr_list]
    model_dict = init_model_dict(num_view, num_class, dim_list, dim_he_list, dim_hvcdn)
    for m in model_dict:
        if cuda:
            model_dict[m].cuda()
    
    print("\nPretrain GCNs...")
    optim_dict = init_optim(num_view, model_dict, lr_e_pretrain, lr_c)
    for epoch in range(num_epoch_pretrain):
        train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                    onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict, train_VCDN=False)
    print("\nTraining...")
    optim_dict = init_optim(num_view, model_dict, lr_e, lr_c)
    for epoch in range(num_epoch+1):
        train_epoch(data_tr_list, adj_tr_list, labels_tr_tensor, 
                    onehot_labels_tr_tensor, sample_weight_tr, model_dict, optim_dict)
        if epoch % test_inverval == 0:
            te_prob = test_epoch(data_trte_list, adj_te_list, trte_idx["te"], model_dict)
            print("\nTest: Epoch {:d}".format(epoch))
            if num_class == 2:
                print("Test ACC: {:.3f}".format(accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test F1: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test AUC: {:.3f}".format(roc_auc_score(labels_trte[trte_idx["te"]], te_prob[:,1])))
            else:
                print("Test ACC: {:.3f}".format(accuracy_score(labels_trte[trte_idx["te"]], te_prob.argmax(1))))
                print("Test F1 weighted: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='weighted')))
                print("Test F1 macro: {:.3f}".format(f1_score(labels_trte[trte_idx["te"]], te_prob.argmax(1), average='macro')))
            print()

In [32]:
cuda = True if torch.cuda.is_available() else False

In [33]:
if __name__ == "__main__":    
    data_folder = 'ROSMAP'
    view_list = [1,2,3]
    num_epoch_pretrain = 500
    num_epoch = 2500
    lr_e_pretrain = 1e-3  
    lr_e = 5e-4
    lr_c = 1e-3
    
    if data_folder == 'ROSMAP':
        num_class = 2
        # adj_parameter = 5  # Try different values
        # dim_he_list = [200,200,100]
    if data_folder == 'BRCA':
        num_class = 5
    
    adj_metric = "cosine"
    
    train_test(data_folder, view_list, num_class,
               lr_e_pretrain, lr_e, lr_c, 
               num_epoch_pretrain, num_epoch)

Hybrid
COS: 
tensor([[1.0000, 0.9808, 0.9889, 0.9931, 0.9895],
        [0.9808, 1.0000, 0.9598, 0.9779, 0.9591],
        [0.9889, 0.9598, 1.0000, 0.9863, 0.9908],
        [0.9931, 0.9779, 0.9863, 1.0000, 0.9835],
        [0.9895, 0.9591, 0.9908, 0.9835, 1.0000]])
From rbf_distance_torch
RBF: 
tensor([[0.0000, 0.0000, 0.0673, 0.0128, 0.0520],
        [0.0364, 0.0000, 0.1076, 0.0467, 0.0935],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0596, 0.0000, 0.0463],
        [0.0000, 0.0000, 0.0194, 0.0000, 0.0000]])
Alpha:  tensor(0.0772)

Hybrid Similarity Matrix Shape: torch.Size([245, 245])
Hybrid Similarity Values (first 5x5):
tensor([[0.0028, 0.0027, 0.0050, 0.0032, 0.0045],
        [0.0031, 0.0022, 0.0050, 0.0034, 0.0046],
        [0.0039, 0.0038, 0.0039, 0.0039, 0.0039],
        [0.0029, 0.0029, 0.0050, 0.0029, 0.0045],
        [0.0037, 0.0036, 0.0045, 0.0036, 0.0037]])
Hybrid
COS: 
tensor([[1.0000, 0.2824, 0.0000, 0.3812, 0.0000],
        [0.2824, 1.0000,